In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import polars as pl
from polars.functions import col
from stanza.pipeline.core import Pipeline
from stanza.utils.conll import CoNLL, Document

# from label_legends.preprocess import load_data, preprocess
from label_legends.util import CONLL_DIR, ROOT, RESOURCE

RESOURCE

PosixPath('/home/lukas/Programming/uni/nlp-ie-label-legends/resource')

# Download files

In [ ]:
# ! wget --output-document resource/edos_labelled_individual_annotations.csv https://github.com/rewire-online/edos/raw/refs/heads/main/data/edos_labelled_individual_annotations.csv 

# Load file

In [4]:
df = pl.scan_csv(RESOURCE / "edos_labelled_individual_annotations.csv")
df.collect()

rewire_id,text,annotator,label_sexist,label_category,label_vector,split
str,str,i64,str,str,str,str
"""sexism2022_english-0""","""[USER] I wonder what keeps tha…",17,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train"""
"""sexism2022_english-0""","""[USER] I wonder what keeps tha…",2,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train"""
"""sexism2022_english-0""","""[USER] I wonder what keeps tha…",6,"""not sexist""","""none""","""none""","""train"""
"""sexism2022_english-1""","""What do you guys think about f…",17,"""not sexist""","""none""","""none""","""train"""
"""sexism2022_english-1""","""What do you guys think about f…",15,"""not sexist""","""none""","""none""","""train"""
…,…,…,…,…,…,…
"""sexism2022_english-9998""","""Since 1973, America has sancti…",2,"""sexist""","""4. prejudiced discussions""","""4.2 supporting systemic discri…","""test"""
"""sexism2022_english-9998""","""Since 1973, America has sancti…",16,"""not sexist""","""none""","""none""","""test"""
"""sexism2022_english-9999""","""Laura Bates Lol she looks like…",2,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train"""


In [5]:
df.filter( col("split") == "train").collect()

rewire_id,text,annotator,label_sexist,label_category,label_vector,split
str,str,i64,str,str,str,str
"""sexism2022_english-0""","""[USER] I wonder what keeps tha…",17,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train"""
"""sexism2022_english-0""","""[USER] I wonder what keeps tha…",2,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train"""
"""sexism2022_english-0""","""[USER] I wonder what keeps tha…",6,"""not sexist""","""none""","""none""","""train"""
"""sexism2022_english-1""","""What do you guys think about f…",17,"""not sexist""","""none""","""none""","""train"""
"""sexism2022_english-1""","""What do you guys think about f…",15,"""not sexist""","""none""","""none""","""train"""
…,…,…,…,…,…,…
"""sexism2022_english-9995""","""[USER] *nod* i try to balance …",15,"""not sexist""","""none""","""none""","""train"""
"""sexism2022_english-9995""","""[USER] *nod* i try to balance …",14,"""not sexist""","""none""","""none""","""train"""
"""sexism2022_english-9999""","""Laura Bates Lol she looks like…",2,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train"""


In [ ]:
tr, val = holdout()

tr

In [ ]:
transform(tr)

In [ ]:
tr.select("tokens")[0,0]

In [ ]:
v = load_vectorizer() #.transform(["witch"]).toarray()
v.vocabulary_

In [ ]:
tr.select("tokens")[0,0].to_list()


In [ ]:
ids = v.transform(tr.select("tokens")[0,0].to_list()).toarray().nonzero()[1]
ids

In [ ]:
ids_to_tokens(tokens_to_ids(tr.select("tokens")[0,0].to_list()))

In [ ]:
v.transform(["witch", "look", "young"]).toarray().nonzero()

In [ ]:
ids = tokens_to_ids(["witch", "look", "a"])
ids

In [ ]:
tokens = ids_to_tokens(ids)
tokens

In [ ]:
tr

In [ ]:
strip_stopwords(tr).select("text", "tokens")[0].to_dicts()

In [ ]:
vectorize_tokens(transform(tr))

In [ ]:
tokens_to_ids(tr.select("tokens")[0,0].to_list())